In [2]:
library(CluMSID) ### for CluMSID
library(matrixStats) ##### for row medians
library(CAMERA) #### for CAMERA
library(qdapRegex) #### for removing number with a pattern
library(dplyr) ##### to filter df1 based on pc group
library(stringr) #### for removing string with certain characters

In [3]:
path_name <- "/Users/mahnoorzulfiqar/OneDriveUNI/S_CBatchResults/MultipleNEG/"
mzml_files <- list.files("/Users/mahnoorzulfiqar/OneDriveUNI/S_CBatchResults/MultipleNEG/", pattern = "PRM.mzML")
mzml_files <- paste(path_name, mzml_files, sep ="")
mzml_files
qcfile <- list.files("/Users/mahnoorzulfiqar/OneDriveUNI/S_CBatchResults/MultiplePOS/", pattern = "pos.mzML")
qcfile <- paste(path_name, qcfile, sep ="")
qcfile

[1] "/Users/mahnoorzulfiqar/OneDriveUNI/S_CBatchResults/MultipleNEG/DS200309_Scost_QC_70k_neg_PRM.mzML"

In [13]:
for (i in mzml_files) {
    #---------------------------------------------------------------------
    ####extract MS2 data with CluMSID
    #---------------------------------------------------------------------
    ms2list <- extractMS2spectra(i) ###########libraryCluMSID for MS2 extraction
    #---------------------------------------------------------------------
    ####features from inclusion list
    #---------------------------------------------------------------------
    ####UNder the assumption there will be just one inclusion list
    inclusion_list <- read.csv("/Users/mahnoorzulfiqar/OneDriveUNI/S_CBatchResults/MultipleNEG/inclusionlist.CSV")##Read inclusion list
    inclusion_list <- inclusion_list[c("Mass..m.z.", "Polarity", "Start..min.", "End..min.")]##Keep important columns
    ID <- c(1:length(inclusion_list[, "Mass..m.z."])) ##count features
    inclusion_list<- cbind(ID, inclusion_list)
    inclusion_list$ID <- paste("M", inclusion_list$ID, sep ="")##give ID
    inclusion_list <- inclusion_list[!grepl("Positive", inclusion_list$Polarity),]##Only negative!
    inclusion_list$Start..min. <- inclusion_list$Start..min.*60 ##convert to seconds
    inclusion_list$End..min. <- inclusion_list$End..min.*60 ##convert to seconds
    cols <- 4:5 ##define columns with RT
    mat<-as.matrix(inclusion_list[cols]) ##covert df to matrix for matrixStats to read
    rtmed <- rowMedians(mat)###########librarymatrixStats (taking median of RT)
    inclusion_list <- cbind(inclusion_list, rtmed) ##add a column of RT Median
    colnames(inclusion_list)[2] <- "mzmed" ##change col names readable by CluMSID 
    colnames(inclusion_list)[1] <- "name" ##change col names readable by CluMSID 
    inclusion_list <- inclusion_list[c("name", "mzmed", "rtmed")] ##keep important columns
    #---------------------------------------------------------------------
    ####merge MS2 spectrum with CluMSID using features from inclusion list
    #---------------------------------------------------------------------
    featlist2 <- mergeMS2spectra(ms2list, peaktable = inclusion_list, exclude_unmatched = TRUE)###########libraryCluMSID for MS2 merging
    #---------------------------------------------------------------------
    ####create a variable for different folder for each mzml file!
    #---------------------------------------------------------------------
    name_w_path <- as.character(i)
    name_w_path <- str_remove(name_w_path, ".mzML")##add this variable to any folder/file name to make it into a different folder and not re-write (with path)
    name_diff <- str_remove(name_w_path, path_name)##add this variable to any folder/file name to make it into a different folder and not re-write (without path)
    #---------------------------------------------------------------------
    ####create a folder for peaklists from each mzml file!
    #---------------------------------------------------------------------
    folderP <- paste(name_w_path, "_peakfiles", sep ="") ##name variable for folder containing peak lists in txt
    if (!file.exists(folderP)){
    dir.create(folderP) ##create folder
    }
    #---------------------------------------------------------------------
    ####save peaklists in text files and save names of files in a variable 
    #---------------------------------------------------------------------
    #### &&&&&&&
    #---------------------------------------------------------------------
    ####save result name files (IMPORTANT FOR METFRAG) 
    #---------------------------------------------------------------------
    fileNames <- c() ### txt file for MS2 Spectra for each m/z
    resultNames <- c() ### result name for each m/z
    id_list = c() ##id from featlist2
    precursor_list = c()##m/z from featlist2
    rt_list = c()##rt from featlist2
    b <- 0
    for(a in 1:length(featlist2)) {
        peakS <- accessSpectrum(featlist2[[a]]) ##access Spectrum
        b <- b+1
        c <- as.character(b) ## to number the files (result names and peak list files)
        fileN <- paste(folderP,'/Peaks_', c, '.txt', sep = '') ##name of peaklists
        write.table(peakS, fileN, row.names = FALSE, col.names = FALSE) ## write peak lists
        fileNames <- c(fileNames, fileN) ## store
        fileR <- paste(name_diff, 'result_', c, sep = '') ##name of result file names (Metfrag)
        resultNames <- c(resultNames, fileR)## store
        idx <- accessID(featlist2[[a]])##access ID
        id_list <- c(id_list, idx)## store
        precursorx <- accessPrecursor(featlist2[[a]])##access m/z
        precursor_list <- c(precursor_list, precursorx)## store
        rtx <- accessRT(featlist2[[a]])##access rt
        rt_list <- c(rt_list, rtx)## store
    }
    first_list <- cbind(id_list, precursor_list, fileNames, resultNames, rt_list) ##Save all information from CluMSID
    #---------------------------------------------------------------------
    ####Processing with CAMERA
    #---------------------------------------------------------------------
    xs <- xcmsSet(file = i, 
            profmethod = "bin", profparam = list(), lockMassFreq=FALSE,
            mslevel= 1, progressCallback=NULL, polarity="negative",
            scanrange = NULL, BPPARAM = bpparam(),
            stopOnError = TRUE)##read mzML
    an <- xsAnnotate(xs)# Create an xsAnnotate object  
    anF <- groupFWHM(an, perfwhm = 0.6)# Group based on RT
    anI <- findIsotopes(anF, mzabs = 0.01) # Annotate isotopes  
    anIC <- groupCorr(anI, cor_eic_th = 0.75)# Verify grouping 
    anFA <- findAdducts(anIC, polarity="negative")#Annotate adducts
    peaklist <- getPeaklist(anFA)
    final_adduct <-peaklist[c("mz", "mzmin", "mzmax", "rt", "rtmin", "rtmax","isotopes", "adduct", "pcgroup", "into")] ##keep important columns
    #---------------------------------------------------------------------
    ####Merge important information from CluMSID and CAMERA
    #---------------------------------------------------------------------
    adducts <-c() ##adducts from final_adduct
    mz_pre <- c() ##m/z from first_list
    pc <- c() ##pc from final_adduct
    peak_list <- c() ##peak lists from first_file
    result_name <- c()##result names from first_file
    rt_list <- c()##rt from first_file
    into_list <- c()##intensity of m/z from final_adduct
    for (d in first_list[,"precursor_list"]) {
        for (j in 1:length(final_adduct[,"rt"])) {
            if (d<=final_adduct[j, "mzmax"] && final_adduct[j, "mzmin"] <=d){
                if (first_list[which(first_list[,"precursor_list"] == d), "rt_list"]<=final_adduct[j, "rtmax"] && final_adduct[j, "rtmin"] <=first_list[which(first_list[,"precursor_list"] == d), "rt_list"]){
                    #mz_pre
                    pre_mzzz <- final_adduct[j, "mz"]
                    mz_pre <- c(mz_pre, pre_mzzz)
                    #rt_list
                    rts <- final_adduct[j, "rt"]
                    rt_list <- c(rt_list, rts)
                    ##from final_adduct
                    #pc_group
                    pcgroupp <- final_adduct[j, "pcgroup"]
                    pc <- c(pc, pcgroupp)
                    #adducts
                    aa<- as.character(final_adduct[j, "adduct"])
                    adducts <- c(adducts, aa)
                    #into_list
                    int<- as.character(final_adduct[j, "into"])
                    into_list <- c(into_list, int)
                    ##from first_list
                    pre <- which(first_list[,"precursor_list"] == d)
                    #peak_list
                    peaklistx <- as.character(first_list[pre,"fileNames"])
                    peak_list <- c(peak_list, peaklistx)
                    #result_name
                    resultnamex <- as.character(first_list[pre,"resultNames"])
                    result_name <- c(result_name, resultnamex)
                }
            }
        }
    }
    final_file1 <- cbind(mz_pre, rt_list, adducts, pc, peak_list, result_name, into_list)
    final_file1<- unique(final_file1)
    final_file_metfrag <- final_file1
    final_file_sirius <- final_file1
    #---------------------------------------------------------------------
    ####Extracting neutral masses
    #---------------------------------------------------------------------
    #write csv file
    csv_file_name <- paste(name_w_path, 'final_file1.csv', sep ='') 
    write.csv(final_file_metfrag, file = csv_file_name)
    na_file <- read.csv(csv_file_name, na.strings=c("", " "))## add NA in blank spaces
    final_result_metfrag <- na.omit(na_file) ## remove any row with NA
    neutral_masses <- gsub("[^0-9.]", " ",final_result_metfrag[, "adducts"])#replace any ions strings with space
    final_result_metfrag <- cbind(final_result_metfrag, neutral_masses)
    #remove oxidation state numbers
    xc<-strsplit(final_result_metfrag[, "neutral_masses"], split = " +")
    xcc <- rm_number(xc, pattern ='"2"')#####libraryqdapRegex (any number with a 2)
    xccc <- rm_number(xcc, pattern ='"3"')#####libraryqdapRegex (any number with a 3)
    xccc4 <- rm_number(xccc, pattern ='"4"')#####libraryqdapRegex (any number with a 4)
    xccc5 <- rm_number(xccc4, pattern ='"5"')#####libraryqdapRegex (any number with a 5)
    xccc6 <- rm_number(xccc5, pattern ='"6"')#####libraryqdapRegex (any number with a 6)
    xccc8 <- rm_number(xccc6, pattern ='"8"')#####libraryqdapRegex (any number with a 8)
    xccc10 <- rm_number(xccc8, pattern ='"10"')#####libraryqdapRegex (any number with a 10)
    neu_masses<-gsub("[^0-9.]", " ",  xccc10)#remove spaces visually
    final_result_metfrag <- cbind(final_result_metfrag, neu_masses)
    file<- subset(final_result_metfrag, select = -c(neutral_masses))#remove neutral_masses
    file_name_csv <- paste(name_w_path, '_CSVfileMetFrag.csv', sep ='')
    write.csv(file, file_name_csv)
    #---------------------------------------------------------------------
    ####Create folder for MetFrag Results
    #---------------------------------------------------------------------
    folderRMet <- paste(name_w_path, "_METFRAG_resultfiles", sep ="") ##name variable for folder containing peak lists in txt
    if (!file.exists(folderRMet)){
        dir.create(folderRMet) ##create folder
    }
    #---------------------------------------------------------------------
    ####Create Parameter txt files for MetFrag
    #---------------------------------------------------------------------
    db <- c("PubChem", "KEGG")
    parameter_file <- c()
    par <- 0
    for (d in 1:length(file[,"neu_masses"])){
        add_mass <- unlist(regmatches(file[d, "neu_masses"],gregexpr("[[:digit:]]+\\.*[[:digit:]]*",file[d, "neu_masses"]))) ##if two or more masses, each mass taken individually
        for (e in add_mass){
            for (t in db) {
                par <- par+1
                para <- as.character(par)
                filePR <- paste(name_w_path, para, t,"_", as.character(e), "_param.txt", sep = "")
                file.create(filePR)
                file.conn <- file(filePR)
                open(file.conn, open = "at")
                writeLines(paste("PeakListPath = ",as.character(file[d,"peak_list"]),sep=""),con=file.conn)
                writeLines(paste("NeutralPrecursorMass = ",as.character(e),sep=""),con=file.conn)
                writeLines(paste("MetFragDatabaseType = ", t, sep=""),con=file.conn)
                writeLines(paste("DatabaseSearchRelativeMassDeviation = ", as.character(5),sep=""),con=file.conn)
                writeLines(paste("FragmentPeakMatchAbsoluteMassDeviation = ", as.character(0.001),sep=""),con=file.conn)
                writeLines(paste("FragmentPeakMatchRelativeMassDeviation = ", as.character(15),sep=""),con=file.conn)
                writeLines(paste("MetFragScoreTypes = ", "FragmenterScore",sep=""),con=file.conn)
                writeLines(paste("MetFragScoreWeights = ", as.character(1.0), sep=""),con=file.conn)
                writeLines(paste("MetFragCandidateWriter = ", "ExtendedFragmentsXLS",sep=""),con=file.conn)
                writeLines(paste("SampleName = ",as.character(file[d,"result_name"]), "_", as.character(e), "_", "mass", t, sep=""),con=file.conn)
                writeLines(paste("ResultsPath = ", folderRMet, sep=""),con=file.conn)
                writeLines(paste("MetFragPreProcessingCandidateFilter = ", "UnconnectedCompoundFilter",sep=""),con=file.conn)
                writeLines(paste("MetFragPostProcessingCandidateFilter = ", "InChIKeyFilter",sep=""),con=file.conn)
                writeLines(paste("MaximumTreeDepth = ", as.character(2),sep=""),con=file.conn)
                writeLines(paste("NumberThreads = ", as.character(1),sep=""),con=file.conn)
                close(file.conn)
                parameter_file <- c(parameter_file,file.conn)   
            }
        }
    }
    
    #---------------------------------------------------------------------
    ####Run Metfrag
    #---------------------------------------------------------------------
    param_file <- list.files(path_name, pattern = "param.txt")
    for (f in param_file) {
        ss <- paste("java -jar /Users/mahnoorzulfiqar/OneDriveUNI/S_CBatchResults/MultipleNEG/MetFrag2.4.5-CL.jar",f)
        system(ss)
    }
    #---------------------------------------------------------------------
    ####features from same group of compounds combined & separated (SIRIUS)
    #---------------------------------------------------------------------
    final_file_sirius <- final_file_sirius[order(pc),]#order in terms of pcgroup
    #divide single pc group rows and the masses with multiple pc groups
    numm <- c()##first number in pc column
    numm2 <- c() ##second number in pc column
    for (g in 2:(length(final_file_sirius[,"mz_pre"]))){
        if (final_file_sirius[g,"pc"]==final_file_sirius[(g-1),"pc"]) {
            ab <- g
            ba <- (g-1)
            numm <- c(numm, ba)
            numm2<- c(numm2, ab)
        }
    } 
    number <- c(numm, numm2)
    unique(number)
    df1 <- unique(final_file_sirius[c(unique(number)),]) ## dataframe with more m/z per compound
    final_file2 <- final_file_sirius[-c(unique(number)),] ## dataframe with one m/z per compound
    file_name_SF_csv <- paste(name_w_path, '_CSVfileSIRIUSF.csv', sep ='')
    write.csv(final_file2, file_name_SF_csv)
    #---------------------------------------------------------------------
    ####input folder for Sirius
    #---------------------------------------------------------------------
    folder_S <- paste(name_w_path, "_SIRIUS_INPUT", sep ="") ##name variable for folder containing peak lists in txt
    if (!file.exists(folder_S)){
        dir.create(folder_S) ##create folder
    }
    #---------------------------------------------------------------------
    ####mass per one compound input files(SIRIUS)
    #---------------------------------------------------------------------
    ##one mass belongs to one compound!
    parameter_file_S <- c()
    par_S <- 0
    for (h in 1:length(final_file2[,"mz_pre"])) {
        par_S <- par_S+1
        para_S <- as.character(par_S)
        file_S <- paste(folder_S, "/", para_S, "_mz_", as.character(final_file2[h,"mz_pre"]),"_SIRIUS_param.ms", sep = "")
        file.create(file_S)
        file.conn <- file(file_S)
        open(file.conn, open = "at")
        #compound
        writeLines(paste(">compound", final_file2[h,"result_name"],sep=" "),con=file.conn)
        #parentmass
        writeLines(paste(">parentmass", final_file2[h,"mz_pre"],sep=" "),con=file.conn)
        #charge
        writeLines(paste(">charge", "-1" ,sep=" "),con=file.conn)
        #rt
        writeLines(paste(">rt", paste(final_file2[h,"rt_list"], "s", sep =""),sep=" "),con=file.conn)
        #ms1
        writeLines(">ms1",con=file.conn)
        #ms1 peaks
        writeLines(paste(final_file2[h,"mz_pre"], final_file2[h,"into_list"],sep=" "),con=file.conn)
        #ms2 peaks
        writeLines(paste(">collision", as.character(30), "eV",sep=" "),con=file.conn)
        kk<- read.table(final_file2[h,"peak_list"])
        for (k in 1:length(kk[,1])){
            writeLines(paste(as.character(kk[k,1]),as.character(kk[k,2]), collapse =" "), con=file.conn) 
        }   
        close(file.conn)
        parameter_file_S <- c(parameter_file_S,file.conn)
    }
    #---------------------------------------------------------------------
    ####more than one mass per one compound input files(SIRIUS)
    #---------------------------------------------------------------------
    #more masses belong to one compound
    df1 <- as.data.frame(df1)
    file_name_SFDF_csv <- paste(name_w_path, '_CSVfileSIRIUS_df.csv', sep ='')
    write.csv(df1, file_name_SFDF_csv)
    parameter_file_same <- c()
    par_same <- 0
    for (l in unique(df1[,"pc"])) {
        result <- filter(df1, pc == l)
        par_same <- par_same+1
        para_same <- as.character(par_same)
        file_SM <- paste(folder_S, "/", "PC_", para_same, "_pc_", l,"_SIRIUS_param.ms", sep = "")
        file.create(file_SM)
        file.conn <- file(file_SM)
        open(file.conn, open = "at")
        ##Compoundname
        result_name_same <- paste(result[,"result_name"], collapse="")
        writeLines(paste(">compound", result_name_same, sep=" "),con=file.conn)
        ##parentmass
        parentm_int <- max(as.numeric(as.character(result[,"into_list"])))
        parent_mass <- which(result[,"into_list"] == parentm_int)
        writeLines(paste(">parentmass", as.character(result[parent_mass, "mz_pre"]), sep=" "),con=file.conn)
        ##charge
        writeLines(paste(">charge", "-1" ,sep=" "),con=file.conn)
        ##ms1peaks
        writeLines(">ms1merged",con=file.conn)
        writeLines(paste(result[,"mz_pre"], result[,"into_list"],sep=" "),con=file.conn)
        ##ms2peaks
        for (m in 1:length(result[,"peak_list"])) {
            peak_same<- read.table(as.character(result[m,"peak_list"]))
            writeLines(paste(">collision", "30 eV",sep=" "),con=file.conn)
            for (n in 1:length(peak_same[,1])){
                writeLines(paste(as.character(peak_same[n,1]),as.character(peak_same[n,2]), sep =" "), con=file.conn) 
            }
        }
        close(file.conn)
        parameter_file_same <- c(parameter_file_same,file.conn)
    }
    sirius_param_file <- list.files(folder_S, pattern = "SIRIUS_param.ms")
    #---------------------------------------------------------------------
    ####Name output directory
    #---------------------------------------------------------------------
    sirius_param_fileOutput <- str_remove(sirius_param_file, ".ms")
    outputNames <- c()
    for (p in 1:length(sirius_param_fileOutput)){
        fileSR <- paste(sirius_param_fileOutput[p],'.json', sep = '')
        outputNames <- c(outputNames, fileSR)
    }
    sirius_param_file_in_out<- cbind(sirius_param_file,outputNames)
    sirius_param_file_in_out <- as.data.frame(sirius_param_file_in_out)
    #---------------------------------------------------------------------
    ####Run SIRIUS
    #---------------------------------------------------------------------
    for (q in 1:length(sirius_param_file_in_out[,"sirius_param_file"])) {
        ss_S <- paste("sirius --input", paste(folder_S, "/", sirius_param_file_in_out[q,"sirius_param_file"], sep =""), "--output", paste(folder_S, "/", sirius_param_file_in_out[q,"outputNames"], sep =""), "formula --candidates 30 --ppm-max 15 structure --database=ALL canopus")
        system(ss_S)
    }
}

Start grouping after retention time.
Created 96 pseudospectra.
Generating peak matrix!
Run isotope peak annotation
 % finished: 10  20  30  40  50  60  70  80  90  100  
Found isotopes: 451 
Start grouping after correlation.

Calculating peak correlations in 96 Groups... 
 % finished: 10  20  30  40  50  60  70  80  90  100  

Calculating graph cross linking in 96 Groups... 
 % finished: 10  20  30  40  50  60  70  80  90  100  
New number of ps-groups:  544 
xsAnnotate has now 544 groups, instead of 96 

Calculating possible adducts in 544 Groups... 
 % finished: 10  20  30  40  50  60  70  80  90  100  
